# End of Semester Grade Calculator

In [13]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from helpers import read_gradebook_csvs, assignment_titles, component_titles, to_percentages, component_grade, component_grades

Multiple gradescope gradebooks will be merged into a single DataFrame.

Add the files to the list below in reverse chronological order of the final exam open time. Doing so will ensure that if a student opened / viewed an earlier final and a later final (e.g. makeup or INTL) the earlier will count. This also allows for the merging of alternate Gradescopes (e.g. for late submissions). The first non-zero score for a given entry takes highest precedence if there are multiple entries for the same student across many CSVs.

In [14]:
gradebook_csv_files = [
    "./COMP110_Spring_2022_grades.csv",
]

grades = read_gradebook_csvs(gradebook_csv_files)
print(f"{grades.shape[0]} records read")
grades.head(5)

695 records read


,First Name,Last Name,SID,section_name,LS00 - Syllabus and Support Understanding,LS00 - Syllabus and Support Understanding - Max Points,LS00 - Syllabus and Support Understanding - Submission Time,LS00 - Syllabus and Support Understanding - Lateness (H:M:S),EX00 - Hello World,EX00 - Hello World - Max Points,...,QZ03 - VersionB - Lateness (H:M:S),LS29 - Introduction to Recursion,LS29 - Introduction to Recursion - Max Points,LS29 - Introduction to Recursion - Submission Time,LS29 - Introduction to Recursion - Lateness (H:M:S),EX10 - Linked List Utils,EX10 - Linked List Utils - Max Points,EX10 - Linked List Utils - Submission Time,EX10 - Linked List Utils - Lateness (H:M:S),Total Lateness (H:M:S)
Email,,,,,,,,,,,,,,,,,,,,,
amehta1@unc.edu,Aadit,Mehta,730486473,COMP110.002.SP22,48.0,49.0,2022-01-13 07:18:49 -0800,10:19:49,100.0,100.0,...,00:00:00,20.0,20.0,2022-04-19 21:14:59 -0700,00:00:00,100.0,100.0,2022-04-27 11:51:43 -0700,00:00:00,260:20:35
agundu16@live.unc.edu,Aakanksha,Gundu,730316807,COMP110.001.SP22,47.0,49.0,2022-01-11 09:13:59 -0800,00:00:00,100.0,100.0,...,00:00:00,18.0,20.0,2022-04-20 22:41:49 -0700,01:42:49,100.0,100.0,2022-04-28 17:48:12 -0700,20:49:12,22:32:01
ajain1@unc.edu,Aarjav,Jain,730506302,COMP110.001.SP22,47.0,49.0,2022-01-12 17:38:34 -0800,00:00:00,100.0,100.0,...,00:00:00,20.0,20.0,2022-04-19 18:47:29 -0700,00:00:00,105.0,100.0,2022-04-25 13:42:39 -0700,00:00:00,00:00:00
aaibrahim@unc.edu,Abdirahman,Ibrahim,0.0,COMP110.001.SP22,47.0,49.0,2022-01-12 15:59:17 -0800,00:00:00,100.0,100.0,...,00:00:00,0.0,20.0,0.0,00:00:00,0.0,100.0,0.0,00:00:00,00:00:00
abeoseh@live.unc.edu,Abeoseh,Flemister,730367424,COMP110.002.SP22,49.0,49.0,2022-01-10 19:12:47 -0800,00:00:00,100.0,100.0,...,00:00:00,19.0,20.0,2022-04-20 13:07:09 -0700,00:00:00,105.2,100.0,2022-04-25 20:30:33 -0700,00:00:00,00:00:00


## Merge in PIDs, ONYENs, and Sections

Follow through the script in `../roster.ipynb` to get a `roster_merged.csv` file that can serve as a mapping between Connect Carolina, Sakai, and Gradesope.

In [15]:
roster = pd.read_csv("./roster_merged.csv")
roster = roster.set_index("GS Email")
roster.head(5)

,PID,Name,Last,First,ONYEN,CC Email,Section
GS Email,,,,,,,
baaron@unc.edu,730530687,"Aaron, Bryce",Aaron,Bryce,bmaaron,baaron@unc.edu,1
shereena@live.unc.edu,730236715,"Ada, Shereena",Ada,Shereena,shereena,shereena@live.unc.edu,3
sourpat@live.unc.edu,730332883,"Adams, Joseph",Adams,Joseph,sourpat,sourpat@live.unc.edu,1
paajoe@email.unc.edu,730397634,"Adjah, Raymond",Adjah,Raymond,paajoe,paajoe@email.unc.edu,1
a8613@live.unc.edu,730312816,"Agrawal, Avi",Agrawal,Avi,a8613,a8613@live.unc.edu,1


Where there are students in the official roster, but not in the gradebook, add their correct e-mail mapping to email_mappings.

In [16]:
# Loop through all e-mails in CC roster
# Find their entry in grades
# Associate their grade DF row with their PID and section
for email in roster.index:
    try:
        student = grades.loc[email]
    except KeyError:
        print(f"Could not find {email} in grades - add to mappings")
        continue
    grades.loc[email, ["Name"]] = roster.loc[email]["Name"]
    grades.loc[email, ["PID"]] = roster.loc[email]["PID"]
    grades.loc[email, ["ONYEN"]] = roster.loc[email]["ONYEN"]
    grades.loc[email, ["First"]] = roster.loc[email]["First"]
    grades.loc[email, ["Last"]] = roster.loc[email]["Last"]
    grades.loc[email, ["Section"]] = roster.loc[email]["Section"]

# Drop people no longer in an official ConnectCarolina section
grades = grades[~grades["Section"].isnull()].copy()

# Convert PIDs to ints
grades["PID"] = grades["PID"].astype(int)

grades.to_csv("gradescope-with-onyens.csv")

grades

,First Name,Last Name,SID,section_name,LS00 - Syllabus and Support Understanding,LS00 - Syllabus and Support Understanding - Max Points,LS00 - Syllabus and Support Understanding - Submission Time,LS00 - Syllabus and Support Understanding - Lateness (H:M:S),EX00 - Hello World,EX00 - Hello World - Max Points,...,EX10 - Linked List Utils - Max Points,EX10 - Linked List Utils - Submission Time,EX10 - Linked List Utils - Lateness (H:M:S),Total Lateness (H:M:S),Name,PID,ONYEN,First,Last,Section
Email,,,,,,,,,,,,,,,,,,,,,
amehta1@unc.edu,Aadit,Mehta,730486473,COMP110.002.SP22,48.0,49.0,2022-01-13 07:18:49 -0800,10:19:49,100.0,100.0,...,100.0,2022-04-27 11:51:43 -0700,00:00:00,260:20:35,"Mehta, Aadit",730486473,aadit171,Aadit,Mehta,2.0
agundu16@live.unc.edu,Aakanksha,Gundu,730316807,COMP110.001.SP22,47.0,49.0,2022-01-11 09:13:59 -0800,00:00:00,100.0,100.0,...,100.0,2022-04-28 17:48:12 -0700,20:49:12,22:32:01,"Gundu, Aakanksha",730316807,agundu16,Aakanksha,Gundu,1.0
ajain1@unc.edu,Aarjav,Jain,730506302,COMP110.001.SP22,47.0,49.0,2022-01-12 17:38:34 -0800,00:00:00,100.0,100.0,...,100.0,2022-04-25 13:42:39 -0700,00:00:00,00:00:00,"Jain, Aarjav",730506302,aarjav,Aarjav,Jain,1.0
aaibrahim@unc.edu,Abdirahman,Ibrahim,0.0,COMP110.001.SP22,47.0,49.0,2022-01-12 15:59:17 -0800,00:00:00,100.0,100.0,...,100.0,0.0,00:00:00,00:00:00,"Ibrahim, Abdirahman",730509993,abdii,Abdirahman,Ibrahim,1.0
abeoseh@live.unc.edu,Abeoseh,Flemister,730367424,COMP110.002.SP22,49.0,49.0,2022-01-10 19:12:47 -0800,00:00:00,100.0,100.0,...,100.0,2022-04-25 20:30:33 -0700,00:00:00,00:00:00,"Flemister, Abeoseh",730367424,abeoseh,Abeoseh,Flemister,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzhang3@unc.edu,Ziqian,Zhang,0.0,COMP110.002.SP22,49.0,49.0,2022-01-12 04:59:21 -0800,00:00:00,100.0,100.0,...,100.0,2022-04-27 19:16:55 -0700,00:00:00,06:27:31,"Zhang, Ziqian",730526947,ziqian1,Ziqian,Zhang,2.0
zf27@ad.unc.edu,Zixuan,Fan,0.0,COMP110.002.SP22,48.0,49.0,2022-01-11 18:32:26 -0800,00:00:00,100.0,100.0,...,100.0,2022-04-25 17:05:34 -0700,00:00:00,00:14:22,"Fan, Zixuan",730365499,zf27,Zixuan,Fan,2.0
ziye.zhou@unc.edu,Ziye,Zhou,730528622,COMP110.002.SP22,46.0,49.0,2022-01-13 02:12:54 -0800,05:13:54,100.0,100.0,...,100.0,2022-04-27 14:44:43 -0700,00:00:00,696:04:53,"Zhou, Ziye",730528622,zziye,Ziye,Zhou,2.0


## Set readings Max Points to 100

Readings 00 and 02 were graded strangely on GS, where the max points is listed as 0.0, so we need to do a bit of preprossing.

In [17]:
# grades["RD00 - Ethical Algorithms - Max Points"] = 100.0
# grades["RD01 - WOMD - Max Points"] = 100.0
# grades["RD02 - Gender Shades - Max Points"] = 100.0

## Get Percentages for All Assignments

In [18]:
grades_pct = to_percentages(grades)
grades_pct.to_csv("grades.csv")
grades_pct

,Name,PID,Section,ONYEN,LS00,EX00,LS01,LS02,LS03,LS04,...,LS27,RD01,RD02,LS28,TA00,EX08,EX09,QZ03,LS29,EX10
Email,,,,,,,,,,,,,,,,,,,,,
amehta1@unc.edu,"Mehta, Aadit",730486473,2.0,aadit171,0.979592,1.0,1.0,0.916667,1.0,1.000000,...,1.000000,3.333333,NaN,0.918919,0.0,0.93,1.000000,0.86,1.00,1.000
agundu16@live.unc.edu,"Gundu, Aakanksha",730316807,1.0,agundu16,0.959184,1.0,1.0,1.000000,1.0,0.928571,...,1.000000,3.166667,NaN,0.918919,0.0,1.00,1.055556,0.72,0.90,1.000
ajain1@unc.edu,"Jain, Aarjav",730506302,1.0,aarjav,0.959184,1.0,1.0,1.000000,1.0,1.000000,...,0.857143,3.333333,NaN,0.945946,0.0,0.95,1.000000,0.96,1.00,1.050
aaibrahim@unc.edu,"Ibrahim, Abdirahman",730509993,1.0,abdii,0.959184,1.0,1.0,0.916667,1.0,0.000000,...,0.000000,0.000000,NaN,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000
abeoseh@live.unc.edu,"Flemister, Abeoseh",730367424,2.0,abeoseh,1.000000,1.0,0.8,1.000000,1.0,1.000000,...,1.000000,3.166667,NaN,1.000000,0.0,1.03,1.055556,0.94,0.95,1.052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzhang3@unc.edu,"Zhang, Ziqian",730526947,2.0,ziqian1,1.000000,1.0,1.0,0.916667,1.0,1.000000,...,1.000000,2.833333,NaN,0.918919,0.0,0.90,1.055556,0.72,1.00,1.000
zf27@ad.unc.edu,"Fan, Zixuan",730365499,2.0,zf27,0.979592,1.0,1.0,0.916667,1.0,1.000000,...,1.000000,3.000000,NaN,0.945946,0.0,1.00,0.977778,0.76,0.95,1.042
ziye.zhou@unc.edu,"Zhou, Ziye",730528622,2.0,zziye,0.938776,1.0,0.9,1.000000,1.0,1.000000,...,0.714286,3.166667,NaN,0.000000,0.0,0.95,0.900000,0.82,0.00,0.982


## Handle Final Grade Replacement Logic

We had 5 quizzes.  If the final is lower than a quiz score, it will replace a single quiz score.

In [19]:
# grades_copy = grades_pct.copy()

# quizzes = component_titles(grades_copy.columns, "QZ")
# quiz_sum = grades_copy[quizzes].sum(axis=1)
# quiz_low = grades_copy[quizzes].min(axis=1)
# final_score = grades_pct[["FN00"]]["FN00"]

# grades_pct.loc[final_score > quiz_low, "QU_SCORE"] = (quiz_sum - quiz_low + final_score) / 5.0
# grades_pct.loc[final_score <= quiz_low, "QU_SCORE"] = quiz_sum / 5.0
# grades_pct["REPLACED_QUIZ"] = final_score > quiz_low
# grades_pct.to_csv("replace_quiz.csv")
# grades_pct

## Compute the Grades for Each Component of the Course

In [20]:
components = {
    # "Final": {
    #     "prefix": "FN",
    #     "drops": 0,
    #     "weight": 10.0
    # },
    "Quizzes": {
        "prefix": "QZ",
        "drops": 0,
        "weight": 40.0
    },
    # "Readings": {
    #     "prefix": "RD",
    #     "drops": 0,
    #     "weight": 10.0
    # },
    "Lessons": {
        "prefix": "LS",
        "drops": 0,
        "weight": 10.0
    },
    # "Projects": {
    #     "prefix": "PJ",
    #     "drops": 0,
    #     "weight": 6.66667
    # },
    "Exercises": {
        "prefix": "EX",
        "drops": 0,
        "weight": 30.0
    }
}

grades_parts = component_grades(grades_pct, components)
grades_parts = grades_parts.sort_values(["Section", "Total"], ascending=[True, True])
grades_parts.to_csv("components.csv")
grades_parts.head()

,Name,PID,Section,ONYEN,QZ,QZ (Pts),QZ (N),LS,LS (Pts),LS (N),EX,EX (Pts),EX (N),Total
sowens1@unc.edu,"Owens, Simon",730487284,1.0,simon503,0.000000,0.000000,0.0,0.061905,0.619048,2.0,0.000000,0.000000,0.0,0.619048
aaibrahim@unc.edu,"Ibrahim, Abdirahman",730509993,1.0,abdii,0.000000,0.000000,0.0,0.176946,1.769463,7.0,0.090909,2.727273,1.0,4.496736
npope@unc.edu,"Pope, Nijah",730487740,1.0,popenk,0.071429,2.857143,1.0,0.432475,4.324751,17.0,0.026364,0.790909,1.0,7.972803
alazartesfaye@unc.edu,"Tesfaye, Alazar",730440263,1.0,at413,0.178491,7.139658,3.0,0.242812,2.428118,9.0,0.037273,1.118182,2.0,10.685958
eliu2@unc.edu,"Liu, Erick",730476659,1.0,eric8931,0.103571,4.142857,1.0,0.459611,4.596105,17.0,0.090909,2.727273,1.0,11.466235


In [21]:
cutoffs = {
    "F": -0.1,
    "D": 59.5,
    "D+": 66.5,
    "C-": 69.5,
    "C": 72.5,
    "C+": 76.5,
    "B-": 79.5,
    "B": 82.5,
    "B+": 86.5,
    "A-": 89.5,
    "A": 92.5 
}

for letter in cutoffs:
    grades_parts.loc[grades_parts["Total"] >= cutoffs[letter], "Grade"] = letter

grades_parts.loc[grades_parts["FN"] == 0.0, "Grade"] = "FA"
grades_parts.to_csv("grades.csv")
grades_parts

KeyError: 'FN'

## Produce CSVs for 

In [ ]:
grades_parts["Final Exam"] = grades_parts["FN"] * 100.0
grades_parts["Final Exam"] = grades_parts["Final Exam"].round(0)
grades_parts["Course Grade"] = grades_parts["Grade"]

for section in [1, 2]:
    grades_sakai = grades_parts[(grades_parts["Section"] == section) & (grades_parts["Grade"] != "FA")][["ONYEN", "Final Exam", "Course Grade"]]
    grades_sakai.to_csv(f"s{section}-sakai.csv", header=["ID", "Final Exam Score", "Course Grade"], index=False)
    grades_cc = grades_parts[grades_parts["Section"] == section][["PID", "ONYEN", "ONYEN", "Course Grade"]]
    grades_cc.to_csv(f"s{section}-connect-carolina.csv", header=False, index=False)

In [ ]:
grades_parts[grades_parts["Total"] >= 92.5].shape[0] / grades_parts.shape[0]

0.4115586690017513